In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import glob
Root = "/content/drive/MyDrive/Ravdess"
os.chdir(Root)

In [5]:
ls

audio_speech_actors_01-24/


In [6]:

import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

!pip install python_speech_features

from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal

!pip install noisereduce

import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')

import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=ad95840695c3337c7b575bd8f93a6215683d14c81d69b67ffb31701175e2fc0d
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [8]:
os.listdir(path='/content/drive/MyDrive/Ravdess')
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = '/content/drive/MyDrive/Ravdess'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

1440

In [9]:
!pip install SpeechRecognition

import speech_recognition as sr
r=sr.Recognizer()
for file in range(0 , len(listOfFiles) , 1):
    with sr.AudioFile(listOfFiles[file]) as source:
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            print(text)
        except:
            print('error')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 18.4 MB/s eta 0:00:00
dogs are sitting by the door
dogs are sitting by the door
dogs are sitting by the door
dogs are sitting by the door
kids are talking by the door
kids are talking by the door
kids are talking by the door
kids are talking by the door
kids are talking by the door
kids are talking by the door
doctor sitting by the door
dogs are sitting by the door
kids are talking by the door
kids are talking by the door
kids are talking by the door
dogs are sitting by the door
dogs are sitting by the door
kids are talking by the door
dogs are sitting by the door
dogs are sitting by the door
dogs are sitting by the door
kids are talking by the door
kids are talking by the door
kids are talking by the door
kids are talking by the door
dogs are sitting by the door
dogs are sitting by the door
dogs are sitting by the door
kids are talking by the door
kids are talking by the door
dogs are sitting by the door
the kids are talking b

In [38]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [39]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [40]:
import os
import glob
from sklearn.model_selection import train_test_split
import numpy as np

def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob("/content/drive/MyDrive/Ravdess/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [41]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
x_train

array([[-6.98676697e+02,  3.99270744e+01,  9.90968704e-01, ...,
         3.82008830e-06,  3.25972383e-06,  9.47736112e-07],
       [-6.47874634e+02,  4.41548691e+01, -1.16805248e+01, ...,
         2.45946449e-05,  1.61880271e-05,  3.83465112e-06],
       [-5.87672852e+02,  4.05068398e+01, -1.66489086e+01, ...,
         1.80800707e-04,  1.00513760e-04,  6.66036140e-05],
       ...,
       [-6.93024231e+02,  5.46327591e+01,  1.00681849e+01, ...,
         3.62368592e-05,  1.73679909e-05,  6.85253144e-06],
       [-6.78819458e+02,  3.36675415e+01,  5.69774723e+00, ...,
         5.98204351e-05,  6.62004313e-05,  5.13996893e-05],
       [-7.00927124e+02,  3.72501717e+01, -1.51359165e+00, ...,
         1.36875169e-05,  8.43393354e-06,  4.48950686e-06]])

In [ ]:
print((x_train.shape[0], x_test.shape[0]))

(577, 193)


In [ ]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:

model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['calm', 'calm', 'calm', 'fearful', 'disgust', 'disgust', 'happy',
       'happy', 'disgust', 'calm', 'fearful', 'disgust', 'disgust',
       'disgust', 'disgust', 'fearful', 'happy', 'disgust', 'calm',
       'disgust', 'calm', 'disgust', 'disgust', 'disgust', 'happy',
       'disgust', 'fearful', 'happy', 'happy', 'disgust', 'disgust',
       'calm', 'disgust', 'disgust', 'fearful', 'fearful', 'fearful',
       'fearful', 'calm', 'fearful', 'happy', 'calm', 'happy', 'disgust',
       'disgust', 'disgust', 'fearful', 'disgust', 'happy', 'fearful',
       'disgust', 'happy', 'happy', 'disgust', 'fearful', 'disgust',
       'disgust', 'calm', 'happy', 'calm', 'happy', 'disgust', 'disgust',
       'calm', 'fearful', 'calm', 'happy', 'disgust', 'disgust',
       'disgust', 'fearful', 'disgust', 'fearful', 'disgust', 'fearful',
       'disgust', 'fearful', 'disgust', 'disgust', 'fearful', 'disgust',
       'disgust', 'fearful', 'fearful', 'fearful', 'fearful', 'fearful',
       'disg

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.47%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.78571429, 0.71641791, 0.71264368, 0.59259259])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,calm,calm
2,calm,calm
3,disgust,fearful
4,disgust,disgust
5,happy,disgust
6,happy,happy
7,happy,happy
8,calm,disgust
9,calm,calm


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Actual').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Predicted').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Predicted'].value_counts()
    for x_label, grp in _df_2.groupby('Actual')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Actual')
_ = plt.ylabel('Predicted')

In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
from IPython.display import Audio
audio_file_path = '/content/drive/MyDrive/Speech/WhatsApp-Ptt-2023-09-01-at-10.39.40.wav'
Audio(audio_file_path)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_13/03-01-01-01-01-02-13.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['disgust'], dtype='<U7')